# Getting info on eSports players from Liquipedia

In [76]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import http

from time import sleep # in case liquipedia blocks me

In [67]:
players_df = pd.read_html('https://liquipedia.net/dota2/Players_(all)', header=0)[0]
players_df.drop(columns={'Unnamed: 0', 'Links'}, inplace=True)

In [64]:
players_URL = 'https://liquipedia.net/dota2/Players_(all)'

players_links = []

res = requests.get(players_URL, headers=hdr)
tree = BeautifulSoup(res.text, 'html5lib')

for row in tree.find_all("tr"):
    if row.find_all("td") is not None:
        link = row.find_all("td")[1]
        players_links.append(link.a.get('href'))

IndexError: list index out of range

In [106]:
players_df.to_csv('~/dotaMediaTermPaper/data/players_df.csv')

In [68]:
players_df.head()

,ID,Name,Team
0,.Ark,Egor Zhabotinskii,NaN
1,13abyKnight,Jon Andersen,NaN
2,1437,Sivatheeban Sivanathapillai,TNC Tiger
3,290,Zeng Chen,PiPiXia
4,2GD,James Harding,NaN


In [69]:
players_ids = players_df['ID'].tolist()

## Get all meta

do something with IP block -- maybe use API you rude fag

In [81]:
players_meta_data = {}

hdr = { 'User-Agent' : 'sorry for that guys! my term paper is on fire!' }

for player in players_ids:
    player_num = players_ids.index(player)
    players_meta_data[player_num] = {}
    
    res = requests.get('https://www.liquipedia.net/dota2/{player}'.format(player=player))
    tree = BeautifulSoup(res.text, 'html5lib')
    
    players_meta_data[player_num].update({'player_id':player})
    
    # get meta data on teams
    descr_box = tree.find('div', class_='fo-nttax-infobox wiki-bordercolor-light')
    
    if descr_box is not None:
        
        # local name
        if descr_box.find(text='Name:') is not None:
            loc_name_raw = descr_box.find(text='Name:')
            loc_name = loc_name_raw.parent.parent.findNext('div').findNext('div').text
            players_meta_data[player_num].update({'local_name':loc_name})
        else:
            continue

        # name in latin
        if descr_box.find(text='Romanized Name:') is not None:
            name_raw = descr_box.find(text='Romanized Name:')
            name = name_raw.parent.parent.findNext('div').findNext('div').text
            players_meta_data[player_num].update({'name':name})
        else:
            continue

        # birth date
        if descr_box.find(text='Birth:') is not None:
            birth_raw = descr_box.find(text='Birth:')
            birth = birth_raw.parent.parent.findNext('div').findNext('div').text
            players_meta_data[player_num].update({'birth_date':birth})
        else:
            continue

        # current status
        if descr_box.find(text='Status:') is not None:
            status_raw = descr_box.find(text='Status:')
            status = status_raw.parent.parent.findNext('div').findNext('div').text
            players_meta_data[player_num].update({'status':status})
        else:
            continue

        # total earnings
        if descr_box.find(text='Approx. Total Earnings:') is not None:
            earn_raw = descr_box.find(text='Approx. Total Earnings:')
            earn = earn_raw.parent.parent.findNext('div').findNext('div').text
            earn = earn.replace('$','').replace(',','')
            players_meta_data[player_num].update({'total_earn':earn})
        else:
            continue
    else:
        continue
        
    sleep(2)
    

SSLError: HTTPSConnectionPool(host='liquipedia.net', port=443): Max retries exceeded with url: /dota2/Gedrox (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [95]:
players_meta_df = pd.DataFrame.from_dict(players_meta_data, orient='index')

In [96]:
players_meta_df.isnull().sum()

player_id       1
local_name      8
name          154
birth_date    196
status        196
total_earn    197
dtype: int64

## Get only players earnings

In [98]:
players_meta_data = {}

for player in players_ids:
    player_num = players_ids.index(player)
    players_meta_data[player_num] = {}
    
    res = requests.get('https://www.liquipedia.net/dota2/{player}'.format(player=player))
    tree = BeautifulSoup(res.text, 'html5lib')
    
    players_meta_data[player_num].update({'player_id':player})
    
    # get meta data on teams
    descr_box = tree.find('div', class_='fo-nttax-infobox wiki-bordercolor-light')
    
    if descr_box is not None:
        # current status
        if descr_box.find(text='Status:') is not None:
            status_raw = descr_box.find(text='Status:')
            status = status_raw.parent.parent.findNext('div').findNext('div').text
            players_meta_data[player_num].update({'status':status})
        else:
            continue

        # total earnings
        if descr_box.find(text='Approx. Total Earnings:') is not None:
            earn_raw = descr_box.find(text='Approx. Total Earnings:')
            earn = earn_raw.parent.parent.findNext('div').findNext('div').text
            earn = earn.replace('$','').replace(',','')
            players_meta_data[player_num].update({'total_earn':earn})
        else:
            continue
        
    sleep(2)

In [102]:
players_earned_status = pd.DataFrame.from_dict(players_meta_data, orient='index')

In [107]:
players_earned_status.head()

,player_id,status,total_earn
0,.Ark,Active\n,6529
1,13abyKnight,Active\n,101159
2,1437,Active\n,193527
3,290,Active\n,2869
4,2GD,Inactive\n,NaN


In [103]:
players_earned_status.to_csv('~/dotaMediaTermPaper/data/players_earned_status.csv')

merging earnings with names

In [108]:
players_df = pd.merge(players_df, players_earned_status, left_on='ID', right_on='player_id', how='left')

In [109]:
players_df.isnull().sum()

ID              0
Name            3
Team          412
player_id       0
status        344
total_earn    388
dtype: int64

In [110]:
players_df.to_csv('~/dotaMediaTermPaper/data/players_df.csv')